## Plotting Azimuth axis on StarTracker images - 05-Dec-22

Craig Lage

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import astropy.io.fits as pf
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
%matplotlib inline

import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle
from astroquery.simbad import Simbad
from lsst.summit.utils.blindSolving import runImchar, genericCameraHeaderToWcs, CommandLineSolver

In [ ]:
solver = CommandLineSolver(indexFiles='/scratch/mfisherlevine/astrometry_net/broiler.astrometry.net/~dstn/4100',
                           checkInParallel=True)

path = '/project/GenericCamera/101/2022/11/22/'

In [ ]:
# Get the image and solve for the WCS
seqNum = 39
filename = path + f'GC101_O_20221122_{seqNum:06}.fits'
exp = afwImage.ExposureF(filename)
wcs = genericCameraHeaderToWcs(exp)
exp.setWcs(wcs)
results = runImchar(exp, 5, 25)
solverResults = solver.run(exp, results.sourceCat, silent=True)
newWcs = solverResults.wcs
raSolve = newWcs.getSkyOrigin().getRa().asDegrees()
decSolve = newWcs.getSkyOrigin().getDec().asDegrees()

In [ ]:
DIR = "/project/GenericCamera/101/2022/11/22/"
HEAD = "GC101_O_20221122_"
hdulist = pf.open(f"{DIR}{HEAD}0000{seqNum}.fits")
header = hdulist[0].header
time = Time(header["DATE-OBS"])

Alt = 90.0
Az = 180.0
now = Time(datetime.now())
altAz = SkyCoord(AltAz(alt=Alt*u.deg, az=Az*u.deg, obstime=time, location=SIMONYI_LOCATION))
RaDec = altAz.transform_to(ICRS)
spherePoint = SpherePoint(afwAngle(RaDec.ra.rad), afwAngle(RaDec.dec.rad))
point = newWcs.skyToPixel(spherePoint)
print(point)

In [ ]:
azimuth_axis = (1552.4894809, 1260.80584883) # From Adrian Shestakov
separation = np.sqrt((point.x - azimuth_axis[0])**2 + (point.y - azimuth_axis[1])**2)
separation += newWcs.getPixelScale().asArcseconds()

In [ ]:
# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.title("StarTracker Wide Field")
img = plt.imshow(exp.image.array,  vmin=100, vmax=2000, interpolation='Nearest', cmap='gray')
plt.scatter(point.x, point.y, marker='+', color='green', s=200,lw=2)
plt.text(point.x+200, point.y, "Astronomical Zenith", color='green', fontsize=18)
plt.scatter(azimuth_axis[0], azimuth_axis[1], marker='+', color='red', s=200,lw=2)
plt.text(azimuth_axis[0]-1000, azimuth_axis[1], "Azimuth Axis", color='red', fontsize=18)
colorbar(img)
plt.tight_layout(h_pad=1)
plt.savefig("/home/craiglagegit/DATA/StarTracker_Azimuth_Axis_20221122_39.pdf")

In [ ]:
seqNum = 43
filename = path + f'GC101_O_20221122_{seqNum:06}.fits'
exp = afwImage.ExposureF(filename)



# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.title("StarTracker Wide Field")
img = plt.imshow(exp.image.array,  vmin=10, vmax=8000, interpolation='Nearest', cmap='gray')
plt.scatter(point.x, point.y, marker='+', color='green', s=200,lw=2)
plt.text(point.x+200, point.y, "Astronomical Zenith", color='green', fontsize=18)
plt.scatter(azimuth_axis[0], azimuth_axis[1], marker='+', color='red', s=200,lw=2)
plt.text(azimuth_axis[0]-1000, azimuth_axis[1], "Azimuth Axis", color='red', fontsize=18)
plt.text(azimuth_axis[0]-500, azimuth_axis[1]+500, f"Separation = {separation:.2f} arcseconds", color='red', fontsize=18)
colorbar(img)
plt.tight_layout(h_pad=1)
plt.savefig("/home/craiglagegit/DATA/StarTracker_Azimuth_Axis_20221122_43.pdf")

In [ ]:
seqNum = 43
filename = path + f'GC101_O_20221122_{seqNum:06}.fits'
exp = afwImage.ExposureF(filename)

Ncenter = (300, 1800)
Nlength = 500.0
Nlabel = 650.0


# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.title("StarTracker Wide Field 2022112200043")
img = plt.imshow(exp.image.array,  vmin=10, vmax=8000, interpolation='Nearest', cmap='gray')
plt.scatter(point.x, point.y, marker='+', color='green', s=200,lw=2)
plt.text(point.x+200, point.y, "Astronomical Zenith", color='green', fontsize=18)
plt.scatter(azimuth_axis[0], azimuth_axis[1], marker='+', color='red', s=200,lw=2)
plt.text(azimuth_axis[0]-1000, azimuth_axis[1], "Azimuth Axis", color='red', fontsize=18)
plt.text(azimuth_axis[0]-500, azimuth_axis[1]+500, f"Separation = {separation:.2f} arcseconds", color='red', fontsize=18)
# Now plot the N, E arrows as determined by ROTPA
rot = Angle(newWcs.getRelativeRotationToWcs(wcs).asDegrees()*u.deg)
plt.arrow(Ncenter[0],Ncenter[1], -Nlength*np.sin(rot), Nlength*np.cos(rot),\
    color='lightgreen', width = 20)
plt.text(Ncenter[0]-Nlabel*np.sin(rot),Ncenter[1]+Nlabel*np.cos(rot), 'N', \
    color='lightgreen', fontsize=12, weight='bold')
plt.arrow(Ncenter[0],Ncenter[1], Nlength*np.cos(rot), Nlength*np.sin(rot),\
    color='lightgreen', width = 20)
plt.text(Ncenter[0]+Nlabel*np.cos(rot),Ncenter[1]+Nlabel*np.sin(rot), 'E', \
    color='lightgreen', fontsize=12, weight='bold')

colorbar(img)
plt.tight_layout(h_pad=1)
plt.savefig("/home/craiglagegit/DATA/StarTracker_Azimuth_Axis_20221122_43.pdf")

In [ ]:
seqNum = 43
filename = path + f'GC101_O_20221122_{seqNum:06}.fits'
exp = afwImage.ExposureF(filename)

Ncenter = (300, 1800)
Nlength = 500.0
Nlabel = 650.0


# Now plot the data with matplotlib
%matplotlib inline
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.title("StarTracker Wide Field 2022112200043")
img = plt.imshow(exp.image.array,  vmin=10, vmax=8000, interpolation='Nearest', cmap='gray')
plt.scatter(point.x, point.y, marker='+', color='green', s=200,lw=2)
plt.text(point.x+200, point.y, "Astronomical Zenith", color='green', fontsize=18)
plt.scatter(azimuth_axis[0], azimuth_axis[1], marker='+', color='red', s=200,lw=2)
plt.text(azimuth_axis[0]-1000, azimuth_axis[1], "Azimuth Axis", color='red', fontsize=18)
plt.scatter(newWcs.getPixelOrigin().x, newWcs.getPixelOrigin().y, marker='+', color='yellow', s=200,lw=2)
plt.text(newWcs.getPixelOrigin().x-100,newWcs.getPixelOrigin().x-700, "Image center", color='yellow', fontsize=18)

plt.text(azimuth_axis[0]-500, azimuth_axis[1]+500, f"Separation = {separation:.2f} arcseconds", color='red', fontsize=18)
# Now plot the N, E arrows as determined by ROTPA
rot = Angle(newWcs.getRelativeRotationToWcs(wcs).asDegrees()*u.deg)
plt.arrow(Ncenter[0],Ncenter[1], -Nlength*np.sin(rot), Nlength*np.cos(rot),\
    color='lightgreen', width = 20)
plt.text(Ncenter[0]-Nlabel*np.sin(rot),Ncenter[1]+Nlabel*np.cos(rot), 'N', \
    color='lightgreen', fontsize=12, weight='bold')
plt.arrow(Ncenter[0],Ncenter[1], Nlength*np.cos(rot), Nlength*np.sin(rot),\
    color='lightgreen', width = 20)
plt.text(Ncenter[0]+Nlabel*np.cos(rot),Ncenter[1]+Nlabel*np.sin(rot), 'E', \
    color='lightgreen', fontsize=12, weight='bold')

colorbar(img)
plt.tight_layout(h_pad=1)
plt.savefig("/home/craiglagegit/DATA/StarTracker_Azimuth_Axis_Center_20221122_43.pdf")

In [ ]:
newWcs.getPixelOrigin()

In [ ]:
newWcs.getPixelOrigin().x